In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re  #regular expression

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 61 columns):
 #   Column                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                         --------------  -----  
 0   Response ID                                                                                                                                                    396 non-null    int64  
 1   Online search (e.g. Google, Bing):BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?      41 non-null     float64
 2   Online shopping website:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on 

In [4]:
Germany.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform' column named 'Platform_cleaned'
unpivoted_GM['Platform_cleaned'] = unpivoted_GM['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_GM['Platform_cleaned'] = unpivoted_GM['Platform_cleaned'].str.replace(
    ":GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

       Response ID                                           Platform  \
23755        94276  None of the above:ROWENTA:GM - And on which of...   
23756        94287  None of the above:ROWENTA:GM - And on which of...   
23757        94291  None of the above:ROWENTA:GM - And on which of...   
23758        95452  None of the above:ROWENTA:GM - And on which of...   
23759        95582  None of the above:ROWENTA:GM - And on which of...   

                Platform_cleaned  Seen Y/N  
23755  None of the above:ROWENTA       NaN  
23756  None of the above:ROWENTA       NaN  
23757  None of the above:ROWENTA       NaN  
23758  None of the above:ROWENTA       NaN  
23759  None of the above:ROWENTA       NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_GM['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     278
2.0     273
3.0     211
4.0     152
99.0     40
Name: count, dtype: int64


In [8]:
# Drop the original 'Platform_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_GM['Seen Y/N'] = np.where(unpivoted_GM['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_GM['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Seen Y/N'].isna(), 0, unpivoted_GM['Seen Y/N'])))

In [9]:
unpivoted_GM.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78230,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
1,78263,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
2,78268,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
3,78278,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
4,78299,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH


In [10]:
count = unpivoted_GM['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22806
1.0       914
99.0       40
Name: count, dtype: int64


Brand
BOSCH        1980
MIELE        1980
SIEMENS      1980
AEG          1980
SAMSUNG      1980
BAUKNECHT    1980
PHILIPS      1980
LG           1980
DYSON        1980
NEFF         1980
HAIER        1980
ROWENTA      1980
Name: count, dtype: int64


Total number of rows: 23760


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 60 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    int64  
 1   Online search (e.g. Google, Bing):ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?  35 non-null     float64
 2   Online shopping website:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or sho

In [12]:
Sweden.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,None of the above:DYSON:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_SW['Platform_cleaned'] = unpivoted_SW['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_SW['Platform_cleaned'] = unpivoted_SW['Platform_cleaned'].str.replace(
    ":SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

       Response ID                                           Platform  \
23595        95995  Product reviews:ROBOROCK:SW - And on which of ...   
23596        96005  Product reviews:ROBOROCK:SW - And on which of ...   
23597        96010  Product reviews:ROBOROCK:SW - And on which of ...   
23598        96062  Product reviews:ROBOROCK:SW - And on which of ...   
23599        96068  Product reviews:ROBOROCK:SW - And on which of ...   

               Platform_cleaned  Seen Y/N  
23595  Product reviews:ROBOROCK       NaN  
23596  Product reviews:ROBOROCK       NaN  
23597  Product reviews:ROBOROCK       NaN  
23598  Product reviews:ROBOROCK       NaN  
23599  Product reviews:ROBOROCK       NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_SW['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     276
3.0     211
2.0     196
4.0     166
99.0     25
Name: count, dtype: int64


In [16]:
# Drop the original 'Platform_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_SW = unpivoted_SW[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_SW['Seen Y/N'] = np.where(unpivoted_SW['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_SW['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Seen Y/N'].isna(), 0, unpivoted_SW['Seen Y/N'])))

In [17]:
unpivoted_SW.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78236,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
1,78238,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
2,78241,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
3,78246,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
4,78248,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22726
1.0       849
99.0       25
Name: count, dtype: int64


Brand
ELECTROLUX    2000
BOSCH         2000
SAMSUNG       2000
MIELE         2000
PHILIPS       2000
SIEMENS       2000
CYLINDA       2000
WHIRLPOOL     2000
LG            2000
DYSON         2000
AEG           2000
ROBOROCK      1600
Name: count, dtype: int64


Total number of rows: 23600


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 57 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     398 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     47 non-null     float64
 2   Online shopping website:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [20]:
Italy.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78233,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_IT['Platform_cleaned'] = unpivoted_IT['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_IT['Platform_cleaned'] = unpivoted_IT['Platform_cleaned'].str.replace(
    ":IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

       Response ID                                           Platform  \
22283        93279  None of the above:SIEMENS:IT - And on which of...   
22284        93284  None of the above:SIEMENS:IT - And on which of...   
22285        93290  None of the above:SIEMENS:IT - And on which of...   
22286        93341  None of the above:SIEMENS:IT - And on which of...   
22287        93357  None of the above:SIEMENS:IT - And on which of...   

                Platform_cleaned  Seen Y/N  
22283  None of the above:SIEMENS       NaN  
22284  None of the above:SIEMENS       NaN  
22285  None of the above:SIEMENS       NaN  
22286  None of the above:SIEMENS       NaN  
22287  None of the above:SIEMENS       NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_IT['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     368
1.0     298
3.0     289
4.0     250
99.0     13
Name: count, dtype: int64


In [24]:
# Drop the original 'Platform_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_IT = unpivoted_IT[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_IT['Seen Y/N'] = np.where(unpivoted_IT['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_IT['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Seen Y/N'].isna(), 0, unpivoted_IT['Seen Y/N'])))

In [25]:
unpivoted_IT.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78233,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,78237,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,78242,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78300,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
4,78315,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     21070
1.0      1205
99.0       13
Name: count, dtype: int64


Brand
SAMSUNG       1990
LG            1990
ELECTROLUX    1990
DYSON         1990
HAIER         1990
SIEMENS       1990
AEG           1990
BEKO          1990
WHIRLPOOL     1592
BOSCH         1592
HOTPOINT      1592
MIELE         1592
Name: count, dtype: int64


Total number of rows: 22288


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 60 columns):
 #   Column                                                                                                                                                        Non-Null Count  Dtype  
---  ------                                                                                                                                                        --------------  -----  
 0   Response ID                                                                                                                                                   396 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?   44 non-null     float64
 2   Online shopping website:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on li

In [28]:
UK.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78258,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_UK['Platform_cleaned'] = unpivoted_UK['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_UK['Platform_cleaned'] = unpivoted_UK['Platform_cleaned'].str.replace(
    ":UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

       Response ID                                           Platform  \
23359        94064  None of the above:NEFF:UK - And on which of th...   
23360        95433  None of the above:NEFF:UK - And on which of th...   
23361        95591  None of the above:NEFF:UK - And on which of th...   
23362        95639  None of the above:NEFF:UK - And on which of th...   
23363        96098  None of the above:NEFF:UK - And on which of th...   

             Platform_cleaned  Seen Y/N  
23359  None of the above:NEFF       NaN  
23360  None of the above:NEFF       NaN  
23361  None of the above:NEFF       NaN  
23362  None of the above:NEFF       NaN  
23363  None of the above:NEFF       NaN  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_UK['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     389
1.0     289
4.0     232
3.0     230
99.0     40
Name: count, dtype: int64


In [32]:
# Drop the original 'Platform_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_UK = unpivoted_UK[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_UK['Seen Y/N'] = np.where(unpivoted_UK['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_UK['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Seen Y/N'].isna(), 0, unpivoted_UK['Seen Y/N'])))

In [33]:
unpivoted_UK.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78251,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,78258,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
2,78259,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78261,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
4,78262,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22184
1.0      1140
99.0       40
Name: count, dtype: int64


Brand
SAMSUNG     1980
LG          1980
BOSCH       1980
DYSON       1980
MIELE       1980
BEKO        1980
SHARK       1980
ZANUSSI     1980
SIEMENS     1980
AEG         1980
NEFF        1980
HOTPOINT    1584
Name: count, dtype: int64


Total number of rows: 23364


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 57 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     60 non-null     float64
 2   Online shopping website:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [36]:
Poland.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,Product reviews:HAIER:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78295,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_PO['Platform_cleaned'] = unpivoted_PO['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_PO['Platform_cleaned'] = unpivoted_PO['Platform_cleaned'].str.replace(
    ":PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

       Response ID                                           Platform  \
22395        96000  Product reviews:AEG:PO - And on which of these...   
22396        96002  Product reviews:AEG:PO - And on which of these...   
22397        96003  Product reviews:AEG:PO - And on which of these...   
22398        96016  Product reviews:AEG:PO - And on which of these...   
22399        96017  Product reviews:AEG:PO - And on which of these...   

          Platform_cleaned  Seen Y/N  
22395  Product reviews:AEG       NaN  
22396  Product reviews:AEG       NaN  
22397  Product reviews:AEG       NaN  
22398  Product reviews:AEG       NaN  
22399  Product reviews:AEG       NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_PO['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     421
2.0     419
3.0     317
4.0     290
99.0     20
Name: count, dtype: int64


In [40]:
# Drop the original 'Platform_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_PO = unpivoted_PO[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_PO['Seen Y/N'] = np.where(unpivoted_PO['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_PO['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Seen Y/N'].isna(), 0, unpivoted_PO['Seen Y/N'])))

In [41]:
unpivoted_PO.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78295,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,78297,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,78309,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78317,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,78344,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG


In [42]:
count = unpivoted_PO['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     20933
1.0      1447
99.0       20
Name: count, dtype: int64


Brand
SAMSUNG       2000
BOSCH         2000
BEKO          2000
AMICA         2000
HAIER         2000
WHIRLPOOL     2000
LG            2000
DYSON         2000
ELECTROLUX    1600
PHILIPS       1600
MIELE         1600
AEG           1600
Name: count, dtype: int64


Total number of rows: 22400


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 57 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     399 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     128 non-null    float64
 2   Online shopping website:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [44]:
Thailand.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,None of the above:TEFAL:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78252,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78269,1.0,2.0,3.0,4.0,1.0,2.0,3.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_TH['Platform_cleaned'] = unpivoted_TH['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_TH['Platform_cleaned'] = unpivoted_TH['Platform_cleaned'].str.replace(
    ":TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

       Response ID                                           Platform  \
22339        95265  Product reviews:DYSON:TH - And on which of the...   
22340        95350  Product reviews:DYSON:TH - And on which of the...   
22341        95488  Product reviews:DYSON:TH - And on which of the...   
22342        95492  Product reviews:DYSON:TH - And on which of the...   
22343        95815  Product reviews:DYSON:TH - And on which of the...   

            Platform_cleaned  Seen Y/N  
22339  Product reviews:DYSON       NaN  
22340  Product reviews:DYSON       NaN  
22341  Product reviews:DYSON       NaN  
22342  Product reviews:DYSON       NaN  
22343  Product reviews:DYSON       NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_TH['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     1112
1.0      959
3.0      839
4.0      784
99.0      10
Name: count, dtype: int64


In [48]:
# Drop the original 'Platform_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_TH = unpivoted_TH[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_TH['Seen Y/N'] = np.where(unpivoted_TH['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_TH['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Seen Y/N'].isna(), 0, unpivoted_TH['Seen Y/N'])))

In [49]:
unpivoted_TH.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78252,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
1,78269,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
2,78313,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78345,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
4,78363,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [50]:
count = unpivoted_TH['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     18640
1.0      3694
99.0       10
Name: count, dtype: int64


Brand
LG            1995
MITSUBISHI    1995
ELECTROLUX    1995
SHARP         1995
BOSCH         1995
HITACHI       1995
TEFAL         1995
HAIER         1995
SAMSUNG       1596
PANASONIC     1596
PHILIPS       1596
DYSON         1596
Name: count, dtype: int64


Total number of rows: 22344


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 56 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     117 non-null    float64
 2   Online shopping website:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [52]:
Vietnam.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,Product reviews:AQUA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AQUA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_VN['Platform_cleaned'] = unpivoted_VN['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_VN['Platform_cleaned'] = unpivoted_VN['Platform_cleaned'].str.replace(
    ":VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

       Response ID                                           Platform  \
21995        95842  Product reviews:MIDEA:VN - And on which of the...   
21996        95844  Product reviews:MIDEA:VN - And on which of the...   
21997        95845  Product reviews:MIDEA:VN - And on which of the...   
21998        95848  Product reviews:MIDEA:VN - And on which of the...   
21999        95852  Product reviews:MIDEA:VN - And on which of the...   

            Platform_cleaned  Seen Y/N  
21995  Product reviews:MIDEA       NaN  
21996  Product reviews:MIDEA       NaN  
21997  Product reviews:MIDEA       NaN  
21998  Product reviews:MIDEA       NaN  
21999  Product reviews:MIDEA       NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_VN['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     831
1.0     817
3.0     570
4.0     541
99.0      9
Name: count, dtype: int64


In [56]:
# Drop the original 'Platform_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_VN = unpivoted_VN[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_VN['Seen Y/N'] = np.where(unpivoted_VN['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_VN['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Seen Y/N'].isna(), 0, unpivoted_VN['Seen Y/N'])))

In [57]:
unpivoted_VN.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78271,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,78275,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,78287,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78291,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,78358,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG


In [58]:
count = unpivoted_VN['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     19232
1.0      2759
99.0        9
Name: count, dtype: int64


Brand
SAMSUNG       2000
LG            2000
PHILIPS       2000
SHARP         2000
AQUA          2000
XIAOMI        2000
HITACHI       2000
PANASONIC     1600
TOSHIBA       1600
ELECTROLUX    1600
BOSCH         1600
MIDEA         1600
Name: count, dtype: int64


Total number of rows: 22000


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 61 columns):
 #   Column                                                                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                                                                             --------------  -----  
 0   Response ID                                                                                                                                                        397 non-null    int64  
 1   Online search (e.g. Google, Bing):SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?        50 non-null     float64
 2   Online shopping website:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searchi

In [60]:
Australia.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_AU['Platform_cleaned'] = unpivoted_AU['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_AU['Platform_cleaned'] = unpivoted_AU['Platform_cleaned'].str.replace(
    ":AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

       Response ID                                           Platform  \
23815        95482  None of the above:AEG:AU - And on which of the...   
23816        95486  None of the above:AEG:AU - And on which of the...   
23817        95487  None of the above:AEG:AU - And on which of the...   
23818        95489  None of the above:AEG:AU - And on which of the...   
23819        95490  None of the above:AEG:AU - And on which of the...   

            Platform_cleaned  Seen Y/N  
23815  None of the above:AEG       NaN  
23816  None of the above:AEG       NaN  
23817  None of the above:AEG       NaN  
23818  None of the above:AEG       NaN  
23819  None of the above:AEG       NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_AU['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     395
1.0     356
4.0     258
3.0     230
99.0     36
Name: count, dtype: int64


In [64]:
# Drop the original 'Platform_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_AU = unpivoted_AU[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_AU['Seen Y/N'] = np.where(unpivoted_AU['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_AU['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Seen Y/N'].isna(), 0, unpivoted_AU['Seen Y/N'])))

In [65]:
unpivoted_AU.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78293,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,78361,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,78394,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,78406,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,78429,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [66]:
count = unpivoted_AU['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22545
1.0      1239
99.0       36
Name: count, dtype: int64


Brand
SAMSUNG         1985
LG              1985
DYSON           1985
WESTINGHOUSE    1985
BOSCH           1985
HISENSE         1985
MIELE           1985
SMEG            1985
ELECTROLUX      1985
HAIER           1985
AEG             1985
Name: count, dtype: int64


Total number of rows: 23820


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 1985


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          1985
LG               1985
DYSON            1985
WESTINGHOUSE     1985
FISHER&PAYKEL    1985
BOSCH            1985
HISENSE          1985
MIELE            1985
SMEG             1985
ELECTROLUX       1985
HAIER            1985
AEG              1985
Name: count, dtype: int64


Total number of rows: 23820


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     398 non-null    int64  
 1   Online search (e.g. Google, Bing):V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?       17 non-null     float64
 2   Online shopping website:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping

In [71]:
Switzerland.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,78375,1.0,2.0,3.0,NaN,NaN,NaN,2.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
1,78387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_SZ['Platform_cleaned'] = unpivoted_SZ['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_SZ['Platform_cleaned'] = unpivoted_SZ['Platform_cleaned'].str.replace(
    ":SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

       Response ID                                           Platform  \
23875        95956  None of the above:SCHULTHESS:SZ - And on which...   
23876        95963  None of the above:SCHULTHESS:SZ - And on which...   
23877        95971  None of the above:SCHULTHESS:SZ - And on which...   
23878        95973  None of the above:SCHULTHESS:SZ - And on which...   
23879        96100  None of the above:SCHULTHESS:SZ - And on which...   

                   Platform_cleaned  Seen Y/N  
23875  None of the above:SCHULTHESS       NaN  
23876  None of the above:SCHULTHESS       NaN  
23877  None of the above:SCHULTHESS       NaN  
23878  None of the above:SCHULTHESS       NaN  
23879  None of the above:SCHULTHESS       NaN  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_SZ['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     242
1.0     196
3.0     185
4.0     168
99.0     29
Name: count, dtype: int64


In [75]:
# Drop the original 'Platform_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_SZ['Seen Y/N'] = np.where(unpivoted_SZ['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_SZ['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Seen Y/N'].isna(), 0, unpivoted_SZ['Seen Y/N'])))

In [76]:
unpivoted_SZ.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,78375,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",1.0,None
1,78387,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
2,78581,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
3,78735,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
4,78849,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None


In [77]:
count = unpivoted_SZ['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     23060
1.0       791
99.0       29
Name: count, dtype: int64


Brand
ELECTROLUX    1990
MIELE         1990
DYSON         1990
BOSCH         1990
SAMSUNG       1990
SIEMENS       1990
PHILIPS       1990
BAUKNECHT     1990
AEG           1990
LIEBHERR      1990
SCHULTHESS    1990
Name: count, dtype: int64


Total number of rows: 23880


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 1990


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         1990
ELECTROLUX    1990
MIELE         1990
DYSON         1990
BOSCH         1990
SAMSUNG       1990
SIEMENS       1990
PHILIPS       1990
BAUKNECHT     1990
AEG           1990
LIEBHERR      1990
SCHULTHESS    1990
Name: count, dtype: int64


Total number of rows: 23880


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q15 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q17.dropna(subset=['Response ID'], inplace=True)

combined_q15 = combined_q15.sort_values(by='Response ID', ascending=True)

# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q15.xlsx', float_format='%.0f', index=False)
combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q15.xlsx', float_format='%.0f', index=False)

print(combined_q15.head(2))

       Response ID                                           Platform  \
0            78230  Online search (e.g. Google, Bing):BOSCH:GM - A...   
23364        78230  None of the above:ROWENTA:GM - And on which of...   

                        Platform_cleaned  Seen Y/N    Brand  
0      Online search (e.g. Google, Bing)       0.0    BOSCH  
23364                  None of the above       0.0  ROWENTA  


In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q15['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207456 entries, 0 to 116779
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Response ID       207456 non-null  int64  
 1   Platform          207456 non-null  object 
 2   Platform_cleaned  207456 non-null  object 
 3   Seen Y/N          207456 non-null  float64
 4   Brand             207456 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 9.5+ MB


In [84]:
count = combined_q15['Brand'].value_counts()
print(count)

Brand
SAMSUNG          17521
BOSCH            17122
LG               15930
DYSON            15521
AEG              13525
ELECTROLUX       13160
MIELE            13127
PHILIPS          11166
HAIER             9950
SIEMENS           9940
BEKO              5970
WHIRLPOOL         5592
HITACHI           3995
SHARP             3995
BAUKNECHT         3970
NEFF              3960
PANASONIC         3196
HOTPOINT          3176
XIAOMI            2000
AQUA              2000
CYLINDA           2000
AMICA             2000
TEFAL             1995
MITSUBISHI        1995
V ZUG             1990
SCHULTHESS        1990
LIEBHERR          1990
SMEG              1985
FISHER&PAYKEL     1985
HISENSE           1985
WESTINGHOUSE      1985
ROWENTA           1980
SHARK             1980
ZANUSSI           1980
ROBOROCK          1600
TOSHIBA           1600
MIDEA             1600
Name: count, dtype: int64
